In [10]:
from tqdm import tqdm
from functools import reduce
import collections
from collections.abc import Iterable

In [2]:
class MyIterable(object):
    def __init__(self):
        pass
    
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError
        return idx, idx*2
it = MyIterable()

In [3]:
for ii in it:
    print(ii)
# looping it again should work
# for i in it:
#     print(i)

(0, 0)
(1, 2)
(2, 4)
(3, 6)
(4, 8)
(5, 10)
(6, 12)
(7, 14)
(8, 16)
(9, 18)


In [4]:
q = 0
prefix = 'Test loop: '
for ii in tqdm(it, desc=prefix, dynamic_ncols=True, leave=True, position=0):
    i1, i2 = ii
    q = i1 + i2
    
print(q)
print(q)
print(q)

Test loop: 100%|██████████| 10/10 [00:00<00:00, 15638.72it/s]

27
27
27


In [18]:
# Check whether something is iterable
def is_iterable(x):
    return isinstance(x, collections.abc.Iterable)

print(is_iterable([]))
print(is_iterable(set()))
print(is_iterable(tuple()))
print(is_iterable({}))
print(is_iterable(map(lambda x: x, [1,2,3])))
print(is_iterable(1))

True
True
True
True
True
False


In [19]:
v = map(lambda x: x, map(lambda x: x, [1,2,3]))
v = list(v)
print(v)

[1, 2, 3]
